#Link Kaggle

In [ ]:
!pip install kaggle
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Imports/Downloads

In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle competitions download -c fake-news

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
submit.csv: Skipping, found more recently modified local copy (use --force to force download)


#Data Preprocessing

In [ ]:
df = pd.read_csv('train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
print(df.shape)
df.head()

(20800, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
df = df.fillna('')
df = df.drop(columns=['title', 'author'])
df.drop_duplicates(subset=['text'],keep='first',inplace=True)
df = df.copy().reset_index()

labels = df.label
labels.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

#Train Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], labels, test_size = 0.2, random_state = 7)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', max_df = 0.7)

tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [ ]:
pac = PassiveAggressiveClassifier(max_iter = 50)
pac.fit(tfidf_train, y_train)

y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100, 2)}%')

Accuracy: 96.05%


In [ ]:
# Build confusion matrix. 1: unreliable, 0: reliable
confusion_matrix(y_test, y_pred, labels=[1, 0])

array([[1916,   87],
       [  74, 2001]])

#Install Libraries

In [ ]:
!pip3 install newspaper3k

     |████████████████████████████████| 215kB 5.5MB/s 
     |████████████████████████████████| 92kB 4.8MB/s 
     |████████████████████████████████| 7.4MB 5.1MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=850f658e7b9dc0608e6fb2f2e317c2f58b4e692efb88fa1356b0c929ad276552
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=6827c8bba2d2112b8d27c12b8777cb96a8be014347c30ef02da66d939756bcf9
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3358 sha256=ec4f6581109577de6db1c1a831509bcf04a0b57346d9d8be31c3ea0710047788
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c75

In [ ]:
from newspaper import Article
 
News_URL = "" #@param {type:"string"}
# download and parse article
article = Article(News_URL)
article.download()
article.parse()

#convert
artext =  tfidf_vectorizer.transform(pd.Series(article.text))
print(pac.predict(artext)[0])
if pac.predict(artext)[0] == 0:
  print(News_URL,'Appears to be reliable')
else:
  print(News_URL,'Appears to be unreliable')


0
https://www.nbcnews.com/news/us-news/trump-should-not-be-prosecuted-capitol-riot-incitement-former-fbi-n1254535 Appears to be reliable


In [ ]:
# from google.colab import files

# joblib.dump(pac,'pac.pkl')
# files.download('pac.pkl')